In [2]:
pip install dash

  Obtaining dependency information for dash from https://files.pythonhosted.org/packages/72/ef/d46131f4817f18b329e4fb7c53ba1d31774239d91266a74bccdc932708cc/dash-2.18.2-py3-none-any.whl.metadata
  Obtaining dependency information for dash-html-components==2.0.0 from https://files.pythonhosted.org/packages/75/65/1b16b853844ef59b2742a7de74a598f376ac0ab581f0dcc34db294e5c90e/dash_html_components-2.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for dash-core-components==2.0.0 from https://files.pythonhosted.org/packages/00/9e/a29f726e84e531a36d56cff187e61d8c96d2cc253c5bcef9a7695acb7e6a/dash_core_components-2.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for dash-table==5.0.0 from https://files.pythonhosted.org/packages/da/ce/43f77dc8e7bbad02a9f88d07bf794eaf68359df756a28bb9f2f78e255bb1/dash_table-5.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for retrying from https://files.pythonhosted.org/packages/8f/04/9e36f28be4c0532c0e9207ff9dc01fb

In [37]:
# import packages

import pandas as pd
import csv
import math
import plotly
import dash
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

In [2]:
# read in datasets

vdem = pd.read_csv('/Users/alecasillas/Desktop/COURSES/STA4930/datasets/V-Dem-CY-Full+Others-v14.csv', encoding= 'utf-8')
bti = pd.read_csv('/Users/alecasillas/Desktop/COURSES/STA4930/datasets/free-fair-elections-bti.csv', quoting=csv.QUOTE_NONE, encoding= 'utf-8')
fiw = pd.read_excel('/Users/alecasillas/Desktop/COURSES/STA4930/datasets/Country_and_Territory_Ratings_and_Statuses_FIW_1973-2024.xlsx', sheet_name = 'Country Ratings, Statuses ', skiprows = 2)
fiw_years = pd.read_excel('/Users/alecasillas/Desktop/COURSES/STA4930/datasets/Country_and_Territory_Ratings_and_Statuses_FIW_1973-2024.xlsx', sheet_name = 'Country Ratings, Statuses ', skiprows=1, nrows=1, header = None)
wrp = pd.read_csv('/Users/alecasillas/Desktop/COURSES/STA4930/datasets/mutated_NWR.csv', quoting=csv.QUOTE_NONE, encoding= 'utf-8')

/var/folders/8s/g7rdmd6n0578b6r7knmss1gr0000gn/T/ipykernel_65651/2661789795.py:3: DtypeWarning: Columns (364,365,366,399,415,804,836,837,924,1240,1257,1486,3094,3168,3169,3341,3342,3344,3345,3347,3350,3352) have mixed types. Specify dtype option on import or set low_memory=False.
  vdem = pd.read_csv('/Users/alecasillas/Desktop/COURSES/STA4930/datasets/V-Dem-CY-Full+Others-v14.csv', encoding= 'utf-8')


In [3]:
# Remove uneccesary quotation marks from the vdem and wrp datasets

vdem = vdem.replace('"', '', regex=True)
new_headers_v = []
for header in vdem.columns: 
    header = header.strip('"') 
    new_headers_v.append(header)
vdem.columns = new_headers_v
new_headers_w = []
for header in wrp.columns: 
    header = header.strip('"') 
    new_headers_w.append(header)
wrp.columns = new_headers_w

In [9]:
# Reshape the FIW dataset to align with the format of the other datasets

PR = [col for col in fiw.columns if col.startswith('PR')]
CL = [col for col in fiw.columns if col.startswith('CL')]
Status = [col for col in fiw.columns if col.startswith('Status')]
fiw_r = pd.lreshape(fiw, {'PR':PR, 'CL':CL, 'Status':Status})

In [10]:
# Align country names to match among all datasets in the appropriate years

country_rename = {
    "Burma/Myanmar": "Myanmar (Burma)",
    "Myanmar": "Myanmar (Burma)",
    "Republic of Vietnam": "South Vietnam",
    "Vietnam, S.": "South Vietnam",
    "Vietnam, N.": "North Vietnam", # add code that says if its before a certain year, change some Vietnams to North Vietnam
    "Yemen, S.": "South Yemen",
    "Yemen, N.": "North Yemen", # add code that says if its before a certain year, change some Yemens to North Yemen
    "Republic of the Congo": "Congo (Brazzaville)",
    "Congo": "Congo (Brazzaville)",
    "Democratic Republic of the Congo ": "Congo (Kinshasa)",
    "Germany, W.": "West Germany", # add code that says if its before a certain year, change some Germanies to West Germany
    "Germany, E.": "East Germany", 
    "German Democratic Republic": "East Germany",
    "Czechia": "Czech Republic", # add code that says if its before a certain year, change it to Czechoslovakia
    "United States": "United States of America",
    "Cote d'Ivoire": "Ivory Coast",
    "Cabo Verde": "Cape Verde",
    "T√ºrkiye": "Turkey",
    "East Timor": "Timor-Leste",
    "The Gambia": "Gambia"
}

vdem["country_name"] = vdem["country_name"].replace(country_rename) 
bti["Entity"] = bti["Entity"].replace(country_rename) 
fiw_r["Country"] = fiw_r["Country"].replace(country_rename) 


for i in range(len(vdem)):
    if (vdem.loc[i,'country_name'] == 'Vietnam') and (1945 < vdem.loc[i,'year'] <= 1976):
        vdem.loc[i,'country_name'] = 'North Vietnam'
    if (vdem.loc[i,'country_name'] == 'Yemen') and (1918 < vdem.loc[i,'year'] <= 1990):
        vdem.loc[i,'country_name'] = 'North Yemen'
    if (vdem.loc[i,'country_name'] == 'Germany') and (1949 < vdem.loc[i,'year'] <= 1990):
        vdem.loc[i,'country_name'] = 'West Germany'
    if (vdem.loc[i,'country_name'] == 'Czech Republic') and (vdem.loc[i,'year'] < 1993):
        vdem.loc[i,'country_name'] = 'Czechoslovakia'

In [10]:
vdem

,country_name,country_text_id,country_id,year,historical_date,project,historical,histname,codingstart,codingend,...,e_mipopula,e_miurbani,e_pefeliex,e_wb_pop,e_pechmor,e_miinteco,e_civil_war,e_miinterc,e_pt_coup,e_pt_coup_attempts
0,Mexico,MEX,3,1789,1789-12-31,1,1,Viceroyalty of New Spain,1789,2023,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN
1,Mexico,MEX,3,1790,1790-12-31,1,1,Viceroyalty of New Spain,1789,2023,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN
2,Mexico,MEX,3,1791,1791-12-31,1,1,Viceroyalty of New Spain,1789,2023,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN
3,Mexico,MEX,3,1792,1792-12-31,1,1,Viceroyalty of New Spain,1789,2023,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN
4,Mexico,MEX,3,1793,1793-12-31,1,1,Viceroyalty of New Spain,1789,2023,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27729,Piedmont-Sardinia,SPD,373,1857,1857-12-31,1,1,Kingdom of Piedmont-Sardinia,1789,1861,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27730,Piedmont-Sardinia,SPD,373,1858,1858-12-31,1,1,Kingdom of Piedmont-Sardinia,1789,1861,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27731,Piedmont-Sardinia,SPD,373,1859,1859-12-31,1,1,Kingdom of Piedmont-Sardinia [incorporates Lom...,1789,1861,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27732,Piedmont-Sardinia,SPD,373,1860,1860-12-31,1,1,NaN,1789,1861,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Merge vdem and bti datasets

data = vdem.merge(bti[['Entity', 'Year', 'Free and fair elections']], left_on = ['country_name', 'year'], right_on = ['Entity', 'Year'], how = 'left')


In [12]:
# Insert Year column back into FIW dataset

fiw_r.insert(1, 'Year', '')

In [13]:
# Reshape FIW Years dataset to prepare for merging

fiw_years = fiw_years.transpose()
fiw_years.columns = fiw_years.iloc[0]  
fiw_years = fiw_years[1:]

In [14]:
# Remove NAs from FIW years dataset

fiw_years.dropna(inplace=True)

In [15]:
# Insert FIW years data back into reshaped FIW dataset

mult = 1
i = 0
while i < len(fiw_r):
    while i < mult*205:
        fiw_r.loc[i, 'Year'] = fiw_years.iloc[mult-1, 0]
        i += 1
    mult += 1

In [16]:
# Merge FIW dataset into master dataset

# maybe dont left join, fiw has data on smaller countries that v-dem doesnt
data = data.merge(fiw_r, left_on = ['country_name', 'year'], right_on = ['Country', 'Year'], how = 'left')

In [17]:
# Merge WRP dataset into master dataset

data = data.merge(wrp, left_on = ['COWcode', 'year'], right_on = ['state', 'year'], how = 'left')

In [18]:
data = data.drop(columns=data.loc[:, 'chrstprot':'pop'].columns)
data = data.drop(columns=data.loc[:, 'datatype':'sourcecode'].columns)
data = data.drop(columns=data.loc[:, 'historical_date':'gap_index'].columns)
data = data.drop(columns=['Entity', 'Year_x', 'Country', 'Year_y', 'state', 'name'])

In [19]:
# Write master dataset to csv

data = data[data['year'] >= 1900]
data.reset_index(drop=True, inplace=True)
with open('capstone_data.csv', 'w+', newline='') as file:
   data.to_csv('capstone_data.csv')

In [20]:
for i in range(len(data)):
    if (data.loc[i,'country_name'] == 'Russia') and (1922 < data.loc[i,'year'] <= 1991):
        data.loc[i,'country_name'] = 'USSR'
    if (data.loc[i,'country_name'] == 'United States of America') and (data.loc[i,'year'] >= 1914):
        data.loc[i,'country_name'] = 'United States'

In [21]:
data

,country_name,country_text_id,country_id,year,COWcode,v2x_polyarchy,v2x_polyarchy_codelow,v2x_polyarchy_codehigh,v2x_polyarchy_sd,v2x_libdem,...,jaingenpct,confgenpct,syncgenpct,anmgenpct,nonreligpct,othrgenpct,sumreligpct,total,dualrelig,reliabilevel
0,Mexico,MEX,3,1900,70.0,0.124,0.112,0.135,0.013,0.060,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Mexico,MEX,3,1901,70.0,0.109,0.096,0.120,0.013,0.058,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Mexico,MEX,3,1902,70.0,0.109,0.096,0.120,0.013,0.058,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Mexico,MEX,3,1903,70.0,0.109,0.096,0.120,0.013,0.058,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Mexico,MEX,3,1904,70.0,0.109,0.096,0.120,0.013,0.058,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19494,Zanzibar,ZZB,236,2019,NaN,0.265,0.233,0.285,0.027,0.211,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19495,Zanzibar,ZZB,236,2020,NaN,0.262,0.235,0.288,0.027,0.207,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19496,Zanzibar,ZZB,236,2021,NaN,0.273,0.244,0.299,0.030,0.220,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19497,Zanzibar,ZZB,236,2022,NaN,0.284,0.254,0.310,0.030,0.229,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# visualization 

from dash import Dash, dcc, html, Input, Output
import plotly.express as px

app = Dash(__name__)

app.layout = html.Div([
    html.H4('Vdem Indicators'),
    html.P("Select a indicator:"),
    dcc.RadioItems(
        id='indicator', 
        options=["v2x_civlib", "v2x_clphy", "v2x_clpol", "v2x_freexp_altinf", "v2xcl_rol", "v2xeg_eqprotec", "v2x_corr", "v2x_execorr", "v2x_pubcorr", "v2x_rule", "v2xcl_acjst", "v2xcl_prpty", "v2juhcind", "Status"],
        value="v2x_corr",
        inline=True
    ),
    dcc.Graph(id="graph"),
])

@app.callback(
    Output("graph", "figure"), 
    Input("indicator", "value"))
def display_choropleth(indicator):
    df = data 
    fig = px.choropleth(
        df, color=indicator, locations="country_text_id", animation_frame='year',
        projection="natural earth")
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return fig

app.run_server(debug=True)

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.preprocessing import LabelEncoder

In [34]:
encoder = LabelEncoder()
data["status_encoded"] = encoder.fit_transform(Y)
Y_encoded = data["status_encoded"]

In [35]:
#X = sm.add_constant(X)

In [39]:
#model = sm.MNLogit(Y_encoded, X).fit()

In [80]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

In [47]:
imp = pd.read_csv('/Users/alecasillas/Desktop/COURSES/STA4930/datasets/imputed.csv', encoding= 'utf-8')

0        4.5
1        6.5
2        6.5
3        6.5
4        6.5
        ... 
53131    6.0
53132    6.0
53133    5.5
53134    5.5
53135    6.0
Name: rescaled, Length: 53136, dtype: float64

In [55]:
features = ["e_gdppc", "v2eldonate", "v2elpubfin", "v2elembaut", "v2elrgpwr", 
          "v2ellocpwr", "v2psbars", "v2exrescon", "v2cltort", "v2caautmob"]
X = features
y = imp["rescaled"]

In [67]:
np.random.seed(42)
imp = pd.DataFrame({feature: np.random.uniform(0, 100, 100) for feature in features})

# -------------------------------
# Function to compute Freedom House Score
# -------------------------------
def compute_freedom_house_score(custom_values, dataset):
    """
    Normalizes each feature from the provided custom values using the dataset's range,
    computes the average normalized value, and maps it to a Freedom House score between 1 and 7.
    """
    scaled_values = {}
    for feature in features:
        min_val = dataset[feature].min()
        max_val = dataset[feature].max()
        # Use min–max normalization (0 to 1)
        if max_val - min_val != 0:
            scaled = (custom_values[feature] - min_val) / (max_val - min_val)
        else:
            scaled = 0.5  # Default value if no variation exists
        scaled_values[feature] = scaled
    
    # Average the normalized values
    avg_scaled = np.mean(list(scaled_values.values()))
    # Map average normalized value (0-1) to Freedom House score (1-7)
    freedom_house_score = 1 + avg_scaled * 6
    return freedom_house_score, scaled_values

# -------------------------------
# Interactive update function
# -------------------------------
def update_freedom_house_score(e_gdppc, v2eldonate, v2elpubfin, v2elembaut,
                               v2elrgpwr, v2ellocpwr, v2psbars, v2exrescon,
                               v2cltort, v2caautmob):
    # Gather custom feature values from sliders
    custom_values = {
        "e_gdppc": e_gdppc,
        "v2eldonate": v2eldonate,
        "v2elpubfin": v2elpubfin,
        "v2elembaut": v2elembaut,
        "v2elrgpwr": v2elrgpwr,
        "v2ellocpwr": v2ellocpwr,
        "v2psbars": v2psbars,
        "v2exrescon": v2exrescon,
        "v2cltort": v2cltort,
        "v2caautmob": v2caautmob
    }
    
    # Compute the Freedom House score and normalized feature values
    score, scaled_vals = compute_freedom_house_score(custom_values, imp)
    
    # Display the current feature values and their normalized (scaled) values
    print("Adjusted Feature Values:")
    for key, value in custom_values.items():
        print(f"  {key}: {value}")
    
    print("\nNormalized Feature Values (0 to 1):")
    for key, value in scaled_vals.items():
        print(f"  {key}_scaled: {value:.2f}")
    
    print(f"\nFreedom House Designation Score (1 to 7): {score:.2f}")
    
    # Visualization: Display a simple bar showing the mapping
    avg_scaled = np.mean(list(scaled_vals.values()))
    fig, ax = plt.subplots(figsize=(6, 1.5))
    
    # Create a horizontal bar chart for the average normalized value
    ax.barh(0, avg_scaled, height=0.4, color='skyblue')
    ax.set_xlim(0, 1)
    ax.set_yticks([])
    ax.set_xlabel("Average Normalized Value (0 to 1)")
    ax.set_title("Mapping to Freedom House Score")
    
    # Annotate the bar with the corresponding Freedom House score
    mapped_value = 1 + avg_scaled * 6
    ax.text(avg_scaled + 0.02, 0, f"Score: {mapped_value:.2f}", va='center', fontsize=12)
    plt.show()


In [68]:
interactive_ui = widgets.interactive(update_freedom_house_score,
    e_gdppc=widgets.FloatSlider(min=imp["e_gdppc"].min(), max=imp["e_gdppc"].max(), step=1, value=imp["e_gdppc"].mean()),
    v2eldonate=widgets.FloatSlider(min=imp["v2eldonate"].min(), max=imp["v2eldonate"].max(), step=1, value=imp["v2eldonate"].mean()),
    v2elpubfin=widgets.FloatSlider(min=imp["v2elpubfin"].min(), max=imp["v2elpubfin"].max(), step=1, value=imp["v2elpubfin"].mean()),
    v2elembaut=widgets.FloatSlider(min=imp["v2elembaut"].min(), max=imp["v2elembaut"].max(), step=1, value=imp["v2elembaut"].mean()),
    v2elrgpwr=widgets.FloatSlider(min=imp["v2elrgpwr"].min(), max=imp["v2elrgpwr"].max(), step=1, value=imp["v2elrgpwr"].mean()),
    v2ellocpwr=widgets.FloatSlider(min=imp["v2ellocpwr"].min(), max=imp["v2ellocpwr"].max(), step=1, value=imp["v2ellocpwr"].mean()),
    v2psbars=widgets.FloatSlider(min=imp["v2psbars"].min(), max=imp["v2psbars"].max(), step=1, value=imp["v2psbars"].mean()),
    v2exrescon=widgets.FloatSlider(min=imp["v2exrescon"].min(), max=imp["v2exrescon"].max(), step=1, value=imp["v2exrescon"].mean()),
    v2cltort=widgets.FloatSlider(min=imp["v2cltort"].min(), max=imp["v2cltort"].max(), step=1, value=imp["v2cltort"].mean()),
    v2caautmob=widgets.FloatSlider(min=imp["v2caautmob"].min(), max=imp["v2caautmob"].max(), step=1, value=imp["v2caautmob"].mean())
)

In [69]:
# Display the interactive widget (works in Jupyter environments)
display(interactive_ui)

interactive(children=(FloatSlider(value=47.018074337820934, description='e_gdppc', max=98.68869366005173, min=…

In [70]:
rename_dict = {
    "e_gdppc": "GDP per Capita",
    "v2eldonate": "Disclosure of campaign donations",
    "v2elpubfin": "Public campaign finance",
    "v2elembaut": "EMB autonomy",
    "v2elrgpwr": "Regional offices relative power",
    "v2ellocpwr": "Local offices relative power",
    "v2psbars": "Barriers to parties",
    "v2exrescon": "Executive respects constitution",
    "v2cltort": "Freedom from torture",
    "v2caautmob": "Mobilization for autocracy"
}

# Original feature keys
original_features = list(rename_dict.keys())
# List of new, user-friendly labels (in the same order)
feature_labels = [rename_dict[key] for key in original_features]


In [88]:
np.random.seed(42)
imp = pd.DataFrame({key: np.random.uniform(0, 100, 100) for key in original_features})
imp = imp.rename(columns=rename_dict)

# -------------------------------
# Function to compute Freedom House Score
# -------------------------------
def compute_freedom_house_score(custom_values, dataset):
    """
    For each feature (using its user-friendly label), normalize the custom value based on the dataset's range,
    compute the average normalized value, and map it to a Freedom House score between 1 and 7.
    """
    scaled_values = {}
    for feature in feature_labels:
        min_val = dataset[feature].min()
        max_val = dataset[feature].max()
        # Avoid division by zero
        if max_val - min_val != 0:
            scaled = (custom_values[feature] - min_val) / (max_val - min_val)
        else:
            scaled = 0.5
        scaled_values[feature] = scaled
    
    avg_scaled = np.mean(list(scaled_values.values()))
    freedom_house_score = 1 + avg_scaled * 6  # Maps average (0-1) to 1-7
    return freedom_house_score, scaled_values

# -------------------------------
# Interactive update function with dynamic plots and live summary
# -------------------------------
def update_freedom_house_score(e_gdppc, v2eldonate, v2elpubfin, v2elembaut,
                               v2elrgpwr, v2ellocpwr, v2psbars, v2exrescon,
                               v2cltort, v2caautmob):
    # Map the slider inputs to user-friendly labels
    custom_values = {
        rename_dict["e_gdppc"]: e_gdppc,
        rename_dict["v2eldonate"]: v2eldonate,
        rename_dict["v2elpubfin"]: v2elpubfin,
        rename_dict["v2elembaut"]: v2elembaut,
        rename_dict["v2elrgpwr"]: v2elrgpwr,
        rename_dict["v2ellocpwr"]: v2ellocpwr,
        rename_dict["v2psbars"]: v2psbars,
        rename_dict["v2exrescon"]: v2exrescon,
        rename_dict["v2cltort"]: v2cltort,
        rename_dict["v2caautmob"]: v2caautmob
    }
    
    # Compute the Freedom House score and normalized values
    score, scaled_vals = compute_freedom_house_score(custom_values, imp)
    
    # Create a live summary interpretation based on the score
    if score < 3:
        interpretation = "Not Free: The country's institutional environment is highly restrictive."
    elif score < 5:
        interpretation = "Partly Free: The country shows mixed attributes regarding freedom."
    else:
        interpretation = "Free: The country maintains a relatively open institutional environment."
    
    summary_text = (f"Freedom House Score: {score:.2f}\n"
                    f"Interpretation: {interpretation}")
    
    # Clear previous output for real-time updating
    clear_output(wait=True)
    
    # Display the adjusted feature values
    print("Adjusted Feature Values:")
    for key in original_features:
        new_label = rename_dict[key]
        print(f"  {new_label}: {custom_values[new_label]}")
    
    print("\nNormalized Feature Values (0 to 1):")
    for feature in feature_labels:
        print(f"  {feature}_scaled: {scaled_vals[feature]:.2f}")
    
    print("\n" + summary_text)
    
    # Create dynamic plots: one for average mapping, one for individual features.
    avg_scaled = np.mean(list(scaled_vals.values()))
    
    fig, axs = plt.subplots(1, 2, figsize=(12, 4))
    
    # Plot 1: Horizontal bar showing the average normalized value mapping to Freedom House Score
    axs[0].barh(0, avg_scaled, height=0.4, color='skyblue')
    axs[0].set_xlim(0, 1)
    axs[0].set_yticks([])
    axs[0].set_xlabel("Average Normalized Value (0 to 1)")
    axs[0].set_title("Overall Mapping to Freedom House Score")
    axs[0].text(avg_scaled + 0.02, 0, f"Score: {score:.2f}", va='center', fontsize=12)
    
    # Plot 2: Bar chart for individual normalized feature values
    feature_names = list(scaled_vals.keys())
    feature_values = list(scaled_vals.values())
    axs[1].barh(feature_names, feature_values, color='lightgreen')
    axs[1].set_xlim(0, 1)
    axs[1].set_xlabel("Normalized Value (0 to 1)")
    axs[1].set_title("Feature Contributions")
    
    plt.tight_layout()
    plt.show()


In [86]:
# -------------------------------
# Function to compute Freedom House Score
# -------------------------------
def compute_freedom_house_score(custom_values, dataset):
    """
    For each feature (using its user-friendly label), normalize the custom value based on the dataset's range,
    compute the average normalized value, and map it to a Freedom House score between 1 and 7.
    """
    scaled_values = {}
    for feature in feature_labels:
        min_val = dataset[feature].min()
        max_val = dataset[feature].max()
        # Avoid division by zero
        if max_val - min_val != 0:
            scaled = (custom_values[feature] - min_val) / (max_val - min_val)
        else:
            scaled = 0.5
        scaled_values[feature] = scaled
    
    avg_scaled = np.mean(list(scaled_values.values()))
    freedom_house_score = 1 + avg_scaled * 6
    return freedom_house_score, scaled_values

# -------------------------------
# Interactive update function
# -------------------------------
def update_freedom_house_score(e_gdppc, v2eldonate, v2elpubfin, v2elembaut,
                               v2elrgpwr, v2ellocpwr, v2psbars, v2exrescon,
                               v2cltort, v2caautmob):
    # Map the inputs to their new, user-friendly labels
    custom_values = {
        rename_dict["e_gdppc"]: e_gdppc,
        rename_dict["v2eldonate"]: v2eldonate,
        rename_dict["v2elpubfin"]: v2elpubfin,
        rename_dict["v2elembaut"]: v2elembaut,
        rename_dict["v2elrgpwr"]: v2elrgpwr,
        rename_dict["v2ellocpwr"]: v2ellocpwr,
        rename_dict["v2psbars"]: v2psbars,
        rename_dict["v2exrescon"]: v2exrescon,
        rename_dict["v2cltort"]: v2cltort,
        rename_dict["v2caautmob"]: v2caautmob
    }
    
    # Compute the Freedom House score and normalized values
    score, scaled_vals = compute_freedom_house_score(custom_values, imp)
    
    # Display the adjusted feature values using the user-friendly labels
    print("Adjusted Feature Values:")
    for old_key in original_features:
        new_label = rename_dict[old_key]
        print(f"  {new_label}: {custom_values[new_label]}")
    
    print("\nNormalized Feature Values (0 to 1):")
    for feature in feature_labels:
        print(f"  {feature}_scaled: {scaled_vals[feature]:.2f}")
    
    print(f"\nFreedom House Designation Score (1 to 7): {score:.2f}")
    
    # Visualization: Horizontal bar chart showing the average normalized value
    avg_scaled = np.mean(list(scaled_vals.values()))
    fig, ax = plt.subplots(figsize=(6, 1.5))
    ax.barh(0, avg_scaled, height=0.4, color='skyblue')
    ax.set_xlim(0, 1)
    ax.set_yticks([])
    ax.set_xlabel("Average Normalized Value (0 to 1)")
    ax.set_title("Mapping to Freedom House Score")
    
    # Annotate with the computed score
    mapped_value = 1 + avg_scaled * 6
    ax.text(avg_scaled + 0.02, 0, f"Score: {mapped_value:.2f}", va='center', fontsize=12)
    plt.show()

# -------------------------------
# Create interactive sliders for each feature
# -------------------------------
interactive_ui = widgets.interactive(update_freedom_house_score,
    e_gdppc=widgets.FloatSlider(
        min=imp[rename_dict["e_gdppc"]].min(), 
        max=imp[rename_dict["e_gdppc"]].max(), 
        step=1, 
        value=imp[rename_dict["e_gdppc"]].mean(),
        description=rename_dict["e_gdppc"]
    ),
    v2eldonate=widgets.FloatSlider(
        min=imp[rename_dict["v2eldonate"]].min(), 
        max=imp[rename_dict["v2eldonate"]].max(), 
        step=1, 
        value=imp[rename_dict["v2eldonate"]].mean(),
        description=rename_dict["v2eldonate"]
    ),
    v2elpubfin=widgets.FloatSlider(
        min=imp[rename_dict["v2elpubfin"]].min(), 
        max=imp[rename_dict["v2elpubfin"]].max(), 
        step=1, 
        value=imp[rename_dict["v2elpubfin"]].mean(),
        description=rename_dict["v2elpubfin"]
    ),
    v2elembaut=widgets.FloatSlider(
        min=imp[rename_dict["v2elembaut"]].min(), 
        max=imp[rename_dict["v2elembaut"]].max(), 
        step=1, 
        value=imp[rename_dict["v2elembaut"]].mean(),
        description=rename_dict["v2elembaut"]
    ),
    v2elrgpwr=widgets.FloatSlider(
        min=imp[rename_dict["v2elrgpwr"]].min(), 
        max=imp[rename_dict["v2elrgpwr"]].max(), 
        step=1, 
        value=imp[rename_dict["v2elrgpwr"]].mean(),
        description=rename_dict["v2elrgpwr"]
    ),
    v2ellocpwr=widgets.FloatSlider(
        min=imp[rename_dict["v2ellocpwr"]].min(), 
        max=imp[rename_dict["v2ellocpwr"]].max(), 
        step=1, 
        value=imp[rename_dict["v2ellocpwr"]].mean(),
        description=rename_dict["v2ellocpwr"]
    ),
    v2psbars=widgets.FloatSlider(
        min=imp[rename_dict["v2psbars"]].min(), 
        max=imp[rename_dict["v2psbars"]].max(), 
        step=1, 
        value=imp[rename_dict["v2psbars"]].mean(),
        description=rename_dict["v2psbars"]
    ),
    v2exrescon=widgets.FloatSlider(
        min=imp[rename_dict["v2exrescon"]].min(), 
        max=imp[rename_dict["v2exrescon"]].max(), 
        step=1, 
        value=imp[rename_dict["v2exrescon"]].mean(),
        description=rename_dict["v2exrescon"]
    ),
    v2cltort=widgets.FloatSlider(
        min=imp[rename_dict["v2cltort"]].min(), 
        max=imp[rename_dict["v2cltort"]].max(), 
        step=1, 
        value=imp[rename_dict["v2cltort"]].mean(),
        description=rename_dict["v2cltort"]
    ),
    v2caautmob=widgets.FloatSlider(
        min=imp[rename_dict["v2caautmob"]].min(), 
        max=imp[rename_dict["v2caautmob"]].max(), 
        step=1, 
        value=imp[rename_dict["v2caautmob"]].mean(),
        description=rename_dict["v2caautmob"]
    )
)

In [87]:
display(interactive_ui)

interactive(children=(FloatSlider(value=47.018074337820934, description='GDP per Capita', max=98.6886936600517…